## Look at the data

In [38]:
#Add the current path into the system path
import sys,os
curPath = os.path.abspath(os.path.dirname("/home/murrawang/"))
sys.path.append(curPath)

# Import necessary libraries
from pyspark.sql import SQLContext
import graphframes
from graphframes import *
from AggregateMessages import *

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql.functions import col, lit, when, avg, collect_list, mean
from pyspark.sql import Row

from datetime import datetime
import numpy as np

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [39]:
#Yelp business data
business_path = "hdfs:///yelp/yelp_academic_dataset_business.json"
yelp_business = spark.read.json(business_path)
# Check which states Yelp concentrates on
yelp_business.select("state").rdd.map(lambda x: (x, 1)).reduceByKey(lambda x,y : x+y).collect()

[(Row(state=u'AZ'), 2),
 (Row(state=u'ON'), 2),
 (Row(state=u'VA'), 2),
 (Row(state=u'NM'), 1),
 (Row(state=u'DE'), 1),
 (Row(state=u'OH'), 11258),
 (Row(state=u'MN'), 1),
 (Row(state=u'AL'), 1),
 (Row(state=u'ME'), 1),
 (Row(state=u'TX'), 24485),
 (Row(state=u'DC'), 1),
 (Row(state=u'NC'), 1),
 (Row(state=u'KY'), 1),
 (Row(state=u'NH'), 4),
 (Row(state=u'FL'), 21907),
 (Row(state=u'CO'), 3198),
 (Row(state=u'KS'), 1),
 (Row(state=u'MI'), 1),
 (Row(state=u'WI'), 1),
 (Row(state=u'IL'), 1),
 (Row(state=u'OR'), 25175),
 (Row(state=u'ABE'), 1),
 (Row(state=u'NY'), 2),
 (Row(state=u'BC'), 17298),
 (Row(state=u'HI'), 1),
 (Row(state=u'GA'), 18090),
 (Row(state=u'WY'), 1),
 (Row(state=u'OK'), 1),
 (Row(state=u'CA'), 13),
 (Row(state=u'WA'), 3121),
 (Row(state=u'MA'), 36012)]

In [40]:
# Here I select businesses in Washington state 
# I drop the businesses which have fewer than 15 reviews to reduce the sparseness.
yelp_business = yelp_business.filter((yelp_business["state"] == "WA") & (yelp_business["review_count"] >= 15))
yelp_business.show(5)

+--------------------+--------------------+--------------------+--------------------+---------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|     city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|        102 E 9th St|[,,,,,,,, False, ...|Yyym58vWvlRPuMCOI...|Local Services, S...|Vancouver|[7:0-17:45, 7:0-1...|      1|45.6282350755|-122.6712514978|Shirley's Tape An...|      98660|          27|  4.5|   WA|
|10722 NW Lakeshor...|[,, u'full_bar', ...|Mf9io8WkMyGautXuS...|Restaurants, Food...|Vancouver|[11:0-21:0, 0:0-0...|      1|   45.6988889|    -122.7

In [41]:
yelp_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [42]:
yelp_business.count()

1589

In [43]:
#Yelp review data
review_path = "hdfs:///yelp/yelp_academic_dataset_review.json"
yelp_review = spark.read.json(review_path)
# Here I only choose reviews on businesses in Washington state
# I apply "dropDuplicates" so that a user only gives one review to a particular business, otherwise our prediction may 
# diverge since the sum of the similarities can exceed one.
yelp_review = yelp_review.join(yelp_business, "business_id", "right").\
                    select("business_id", "user_id", yelp_review.stars).dropDuplicates(["business_id", "user_id"])
yelp_review.show(5)

+--------------------+--------------------+-----+
|         business_id|             user_id|stars|
+--------------------+--------------------+-----+
|-36nnCT71XE0InJXK...|5hjAPltdXiHM3Ng4i...|  1.0|
|-36nnCT71XE0InJXK...|WiVyzMjKiOOyF2kK1...|  5.0|
|-36nnCT71XE0InJXK...|tU52QQnMGbJA8o89z...|  1.0|
|-36nnCT71XE0InJXK...|Xg7RVVk9zNXTXyvCZ...|  1.0|
|-36nnCT71XE0InJXK...|wKBjsz3O0DYzXIxr8...|  2.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [44]:
yelp_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- stars: double (nullable = true)



In [45]:
yelp_review.count()

102383

In [46]:
#save the review dataset
yelp_review.write.save("hdfs:///yelp/yelp_review.json", format="json")

## Loading the small datasets

In [1]:
#Add the current path into the system path
import sys,os
curPath = os.path.abspath(os.path.dirname("/home/murrawang/"))
sys.path.append(curPath)

# Import necessary libraries
from pyspark.sql import SQLContext
import graphframes
from graphframes import *
from AggregateMessages import *

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql.functions import col, lit, when, avg, collect_list, mean
from pyspark.sql import Row

from datetime import datetime
import numpy as np

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [2]:
#loading the target dataset
review_path = "hdfs:///yelp/yelp_review.json"
yelp_review = spark.read.json(review_path)

In [3]:
# split the review data to training and test parts
(training_r, test_r) = yelp_review.randomSplit([0.8, 0.2], seed=11)
training_r.cache()
test_r.cache()
print('Number of partitions: ', training_r.rdd.getNumPartitions())
print('Number of workers:', int(sc._conf.get('spark.executor.instances')))

('Number of partitions: ', 7)
('Number of workers:', 2)


In [4]:
yelp_review.count()

102383

In [5]:
training_r.count()

81978

In [6]:
test_r.count()

20405

In [7]:
# Construct a dataframe for users with necessary information (average ratings and review counts)
yelp_user = yelp_review.select("user_id").distinct()
yelp_user = yelp_user.join(training_r.groupBy("user_id").avg("stars").withColumnRenamed("avg(stars)", "avg_stars"), "user_id", "left").\
                  join(training_r.groupBy("user_id").count().withColumnRenamed("count", "review_count"), "user_id", "left").na.fill(0)

In [8]:
yelp_user.show(5)

+--------------------+---------+------------+
|             user_id|avg_stars|review_count|
+--------------------+---------+------------+
|IJfkwQf8Ad-Q9XKVc...|      3.0|           1|
|p6jrAZRs7bsknyujw...|      5.0|           3|
|U8Jsf4RO5kSAS0ahr...|      5.0|           2|
|DwVgF-OAfqIoBd1fa...|      3.0|           2|
|Zuop-axbyA7atet3p...|      4.0|           1|
+--------------------+---------+------------+
only showing top 5 rows



In [9]:
yelp_user.count()

44780

In [10]:
# Construct a dataframe for businesses with necessary information (average ratings and review counts)
yelp_business = yelp_review.select("business_id").distinct()
yelp_business = yelp_business.join(training_r.groupBy("business_id").avg("stars").withColumnRenamed("avg(stars)", "avg_stars"), "business_id", "left").\
                  join(training_r.groupBy("business_id").count().withColumnRenamed("count", "review_count"), "business_id", "left").na.fill(0)

In [11]:
yelp_business.show(5)

+--------------------+------------------+------------+
|         business_id|         avg_stars|review_count|
+--------------------+------------------+------------+
|-36nnCT71XE0InJXK...|1.9565217391304348|          23|
|18zCSuupFXh6wIygX...| 4.140845070422535|         142|
|3l-DALnAnPVKndBbz...|              3.04|          50|
|OlWIKiqL6wUGjK6Do...|               4.0|          26|
|W3Pv_BO9BShN-4cHe...|3.7777777777777777|          36|
+--------------------+------------------+------------+
only showing top 5 rows



In [12]:
yelp_business.count()

1589

In [13]:
# look at the degree of sparseness
print("Average number of reviews of users: ", float(yelp_review.count())/float(yelp_user.count()))
print("Average number of reviews of businesses: ", float(yelp_review.count())/float(yelp_business.count()))

('Average number of reviews of users: ', 2.2863555158552926)
('Average number of reviews of businesses: ', 64.43234738829453)


## Derive the similarities betweem pairs of businesses

In [14]:
# "U" is the prefix for user id and "B" is the prefix for business id to make the vertex ids unique 
# (i.e. to avoid an user id with the same value of a business id)
bt = yelp_business.rdd.map(lambda r: Row(business_id=r.business_id, total_bstars=r.avg_stars*r.review_count)).toDF()
ut = yelp_user.rdd.map(lambda r: Row(user_id=r.user_id, total_ustars=r.avg_stars*r.review_count)).toDF()

review_with_info = training_r.join(ut, "user_id", "left").join(bt, "business_id", "left")\
                               .rdd.map(lambda r: Row(user_info=["U"+str(r.user_id), r.total_ustars], business_id="B"+str(r.business_id), total_bstars=r.total_bstars, \
                                stars=r.stars)).toDF()

In [15]:
review_with_info.show(5)

+--------------------+-----+------------+--------------------+
|         business_id|stars|total_bstars|           user_info|
+--------------------+-----+------------+--------------------+
|B-36nnCT71XE0InJX...|  1.0|        45.0|[UwQkvbIqEuTOLNg-...|
|B-36nnCT71XE0InJX...|  1.0|        45.0|[UAsG6oQm4msBBHGk...|
|B-36nnCT71XE0InJX...|  1.0|        45.0|[UilvHx5hnf7mnaVw...|
|B-36nnCT71XE0InJX...|  5.0|        45.0|[UI9YnU0WPIDrAZNM...|
|B-36nnCT71XE0InJX...|  1.0|        45.0|[UofyOSbE04KsDt7e...|
+--------------------+-----+------------+--------------------+
only showing top 5 rows



In [16]:
review_with_info.count()

81978

In [17]:
df_as1 = review_with_info.withColumnRenamed("business_id", "b1_id").\
               withColumnRenamed("stars", "b1_stars").withColumnRenamed("total_bstars", "b1_total_bstars")
df_as2 = review_with_info.withColumnRenamed("business_id", "b2_id").\
               withColumnRenamed("stars", "b2_stars").withColumnRenamed("total_bstars", "b2_total_bstars")
bpairs = df_as1.join(df_as2, "user_info")
bpairs.show(5)

+--------------------+--------------------+--------+-----------------+--------------------+--------+-----------------+
|           user_info|               b1_id|b1_stars|  b1_total_bstars|               b2_id|b2_stars|  b2_total_bstars|
+--------------------+--------------------+--------+-----------------+--------------------+--------+-----------------+
|[U-933LjXpSOX57wO...|B7DQjIVYGvcv2g31m...|     5.0|608.9999999999999|B7DQjIVYGvcv2g31m...|     5.0|608.9999999999999|
|[U-DUBZFaADuQxYh1...|BJmyyYki9NlQtjcc6...|     5.0|            352.0|BtJTO7Vlt6Zuke1LO...|     4.0|            387.0|
|[U-DUBZFaADuQxYh1...|BJmyyYki9NlQtjcc6...|     5.0|            352.0|BBDX4VYxx_DpuAEnN...|     4.0|           1052.0|
|[U-DUBZFaADuQxYh1...|BJmyyYki9NlQtjcc6...|     5.0|            352.0|BJmyyYki9NlQtjcc6...|     5.0|            352.0|
|[U-DUBZFaADuQxYh1...|BtJTO7Vlt6Zuke1LO...|     4.0|            387.0|BtJTO7Vlt6Zuke1LO...|     4.0|            387.0|
+--------------------+--------------------+-----

In [18]:
bpairs.count()

681124

In [19]:
# Calculating the recommendation power
def RecommendPower(row):
    rp = (row.b1_stars*row.b2_stars)/(row.b1_total_bstars*float(row.user_info[1]))
    return Row(b1_id=row.b1_id, b2_id=row.b2_id, rp=rp)

In [20]:
bpairs_withrp = bpairs.rdd.map(RecommendPower).toDF()
bpairs_withrp.show(5)

+--------------------+--------------------+--------------------+
|               b1_id|               b2_id|                  rp|
+--------------------+--------------------+--------------------+
|B7DQjIVYGvcv2g31m...|B7DQjIVYGvcv2g31m...|0.008210180623973728|
|BJmyyYki9NlQtjcc6...|BJmyyYki9NlQtjcc6...|0.005463286713286713|
|BJmyyYki9NlQtjcc6...|BBDX4VYxx_DpuAEnN...|0.004370629370629371|
|BJmyyYki9NlQtjcc6...|BtJTO7Vlt6Zuke1LO...|0.004370629370629371|
|BtJTO7Vlt6Zuke1LO...|BJmyyYki9NlQtjcc6...|0.003975352812562...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [21]:
# Now we add all the recommend power for each pair of businesses and get the similarity 
business_similarity = bpairs_withrp.groupBy(["b1_id", "b2_id"]).sum("rp").withColumnRenamed("sum(rp)", "similarity")
business_similarity.show(5)

+--------------------+--------------------+--------------------+
|               b1_id|               b2_id|          similarity|
+--------------------+--------------------+--------------------+
|B7B8xSsrJyaduoLBa...|B5ujD30qvzMUYDkp7...|0.002066400330624053|
|Bh9HSiYDrIiLDiXox...|Bh9HSiYDrIiLDiXox...| 0.25836680637159176|
|BQh477Dag_gSdzaIL...|BBMoxh2zG5lVDADDz...|2.554278416347382E-4|
|B8DyJnIx3zYu3riHg...|B-cmLahbAhCgi-Zw2...|4.191886544827721...|
|BeTjaXgajoYKVt-O3...|Bn0lX7M9A_YqEFq8C...|0.002419538056270...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [22]:
# Note that the number of pairs now is less than 1589^2 because lots of pairs of businesses do not have similarities, i.e. similarity=0.
business_similarity.count()

371653

## Build the Bipartite Graph

In [25]:
# "U" is the prefix for user id and "B" is the prefix for business id to make the vertex ids unique 
# (i.e. to avoid an user id with the same value of a business id)
businessVertices = yelp_business.rdd.map(lambda r: Row(id="B"+str(r.business_id), vtype="business", avg_stars=r.avg_stars, review_count=r.review_count)).toDF()
userVertices = yelp_user.rdd.map(lambda r: Row(id="U"+str(r.user_id), vtype="user", avg_stars=r.avg_stars, review_count=r.review_count)).toDF()

# Pack the businesses' information into edges for the prediction part!
reviewEdges = training_r.join(yelp_user, "user_id", "left")\
                      .rdd.map(lambda r: Row(src="U"+str(r.user_id),dst="B"+str(r.business_id), \
                        bstars=["B"+str(r.business_id), r.stars])).toDF()
    
# build the graph
bipartiteVertices = userVertices.unionAll(businessVertices)
bipartiteGraph = GraphFrame(bipartiteVertices, reviewEdges)

In [26]:
reviewEdges.count()

81978

## The weighted Bipartite Graph Projection

In [27]:
# Add the business_2's average stars to the dataframe for further use
business_similarity = business_similarity.join(businessVertices.select("id", "avg_stars").\
                         withColumnRenamed("id", "b2_id").withColumnRenamed("avg_stars", "b2_avg_stars"), "b2_id", "left")
business_similarity.show(5)

+--------------------+--------------------+--------------------+-----------------+
|               b2_id|               b1_id|          similarity|     b2_avg_stars|
+--------------------+--------------------+--------------------+-----------------+
|B9ptGBKGZ5oF_y-Ab...|BnU-S7XRc14kT1wNM...|0.001564945226917058|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|BYEsYXr38-1sdjroe...|0.001061841658171...|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|BoP4JpYJ56vfJMrMR...| 9.92063492063492E-4|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|BlD-xYJUeOa293piQ...|6.105006105006105E-4|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|BUH0U_RfNjYeNHY0O...|0.002986857825567503|4.636363636363637|
+--------------------+--------------------+--------------------+-----------------+
only showing top 5 rows



In [28]:
# Now we construct the weighted Bipartite Graph Projection for the business set where the weights are similarities between businesses
# Pack the business_2's information into edges for the prediction part!
weightEdges = business_similarity.rdd.map(lambda r: Row(src=r.b1_id, dst=r.b2_id, sim=[r.b2_id, r.b2_avg_stars, r.similarity])).toDF()

# build the graph
BGprojection = GraphFrame(businessVertices, weightEdges)

In [29]:
# Add the prefix "U" and "B" to match the columns in the graph
testr_withprefix = test_r.rdd.map(lambda r: Row(business_id="B"+str(r.business_id), user_id="U"+str(r.user_id), stars=r.stars)).toDF()

In [30]:
# aggregateMessages for similarities
msgtoSrc = AM.edge['sim']
testr_withsim = BGprojection.aggregateMessages\
                            (collect_list(AM.msg).alias('sim_list'), sendToSrc=msgtoSrc).withColumnRenamed("id", "business_id").\
                            join(testr_withprefix, "business_id", "right").\
                            join(businessVertices.select("id", "avg_stars").withColumnRenamed("id", "business_id"), "business_id", "left")

In [31]:
testr_withsim.show(5)

+--------------------+--------------------+-----+--------------------+-----------------+
|         business_id|            sim_list|stars|             user_id|        avg_stars|
+--------------------+--------------------+-----+--------------------+-----------------+
|B9ptGBKGZ5oF_y-Ab...|[[B9ptGBKGZ5oF_y-...|  5.0|U-aT-w7qFF8UsYApn...|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|[[B9ptGBKGZ5oF_y-...|  5.0|UEG0Y2P0QhoHl-Z3_...|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|[[B9ptGBKGZ5oF_y-...|  5.0|UQ8Z6ge1g1UQOms7I...|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|[[B9ptGBKGZ5oF_y-...|  5.0|UYzH5nTagxL2C0Sqa...|4.636363636363637|
|B9ptGBKGZ5oF_y-Ab...|[[B9ptGBKGZ5oF_y-...|  5.0|UtkuxxQHPPHwZgf6i...|4.636363636363637|
+--------------------+--------------------+-----+--------------------+-----------------+
only showing top 5 rows



In [32]:
# aggregateMessages for br_lists
msgtoSrc = AM.edge['bstars']
testr_withsimbr = bipartiteGraph.aggregateMessages\
                            (collect_list(AM.msg).alias('br_list'), sendToSrc=msgtoSrc).withColumnRenamed("id", "user_id").\
                            join(testr_withsim, "user_id", "right")

In [33]:
testr_withsimbr.show(5)

+--------------------+--------------------+--------------------+--------------------+-----+-----------------+
|             user_id|             br_list|         business_id|            sim_list|stars|        avg_stars|
+--------------------+--------------------+--------------------+--------------------+-----+-----------------+
|U0-jGGeSVW9EMS607...|[[BM6Iud4p9KO01KT...|BOtgtzNydhtwNR0EC...|[[BeU-DrRN9ZSWtFX...|  5.0|4.482758620689655|
|U2lMVWQYWb-_Wpj7C...|[[Bs9_U99yktaNqjD...|BRJBCFf7Ov6dRguTr...|[[Bk0KhcOuhXxxrsW...|  2.0|3.871212121212121|
|U2lMVWQYWb-_Wpj7C...|[[Bs9_U99yktaNqjD...|BnoE7D1xm8USfNJY6...|[[Bbzny-7M_JjYHsK...|  3.0|4.212765957446808|
|U2lMVWQYWb-_Wpj7C...|[[Bs9_U99yktaNqjD...|BbObINYi8__KgdWnc...|[[Bvty9cDLZpwSTkH...|  3.0|              4.4|
|U2m6cfArIoaGquQuM...|                null|BEeAczDPkCXbjHt2R...|[[BHGWLckLMDjP4hB...|  4.0|4.232558139534884|
+--------------------+--------------------+--------------------+--------------------+-----+-----------------+
only showi

In [34]:
# predict the ratings
def Prediction(row):
    pred = row.avg_stars
    if (row.sim_list is not None and row.br_list is not None):
        for i in range(len(row.sim_list)):
            for j in range(len(row.br_list)):
                if row.sim_list[i][0] == row.br_list[j][0]:
                    pred += float(row.sim_list[i][2])*(float(row.br_list[j][1])-float(row.sim_list[i][1]))
    return Row(business_id=row.business_id, user_id=row.user_id, actual_stars=row.stars, pred_stars=pred)

In [35]:
#Finally, we get the predictions.
result = testr_withsimbr.rdd.map(Prediction).toDF()
result.show(5)

+------------+--------------------+-----------------+--------------------+
|actual_stars|         business_id|       pred_stars|             user_id|
+------------+--------------------+-----------------+--------------------+
|         5.0|BOtgtzNydhtwNR0EC...|4.482758620689655|U0-jGGeSVW9EMS607...|
|         3.0|BbObINYi8__KgdWnc...|4.388206986694272|U2lMVWQYWb-_Wpj7C...|
|         2.0|BRJBCFf7Ov6dRguTr...|3.869973103784599|U2lMVWQYWb-_Wpj7C...|
|         3.0|BnoE7D1xm8USfNJY6...|4.219260264248601|U2lMVWQYWb-_Wpj7C...|
|         4.0|BEeAczDPkCXbjHt2R...|4.232558139534884|U2m6cfArIoaGquQuM...|
+------------+--------------------+-----------------+--------------------+
only showing top 5 rows



In [36]:
# evaluate the RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="actual_stars", predictionCol="pred_stars")
rmse = evaluator.evaluate(result)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.41038292864
